<a href="https://colab.research.google.com/github/GCHeroes1/GDC/blob/master/Parallelise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
np.random.seed(0)
import torch
torch.cuda.empty_cache
torch.manual_seed(0)
import random
random.seed(0)
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import Module
from torch.autograd import Variable
from matplotlib import pyplot as plt
import cv2
import os
import torchvision
import PIL
import math
import pandas as pd
import shutil

In [2]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!apt update && apt install -y openslide-tools
!pip install openslide-python

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [5]:
import openslide
import openslide.deepzoom

In [6]:
df = pd.read_csv("/content/drive/MyDrive/reconstructedData.csv")
df = df[df['slides__file_id'].notna()]
df = df[df['years_survived'].notna()]
df

,patient_id,vital_status,years_to_death,age_at_diagnosis,year_of_diagnosis,tumor_stage,tissue_or_organ_of_origin,site_of_resection_or_biopsy,biopsy_tissue_type,prior_malignancy,primary_diagnosis,years_survived,slides__slide_id,slides__file_id,slides__file_size,slides__case_id,slides__section_location,slides__submitter_id,slides__SampleCode,slides__sample_type,slides__percent_stromal_cells,slides__percent_tumor_cells,slides__percent_neutrophil_infiltration,slides__percent_lymphocyte_infiltration,slides__percent_necrosis,slides__percent_normal_cells,slides__percent_monocyte_infiltration,slides__percent_tumor_nuclei,years_to_last_follow_up
0,9817ec15-605a-40db-b848-2199e5ccbb7b,Dead,3.506849,63.873973,1994.0,stage ii,"Skin, NOS",Lymph nodes of axilla or arm,Lymph,no,"Malignant melanoma, NOS",3.506849,b90d346a-e8b9-4e71-ada7-bce4cd3248da,71208712-2893-4404-9cef-ff090774d057,8.755867e+08,9817ec15-605a-40db-b848-2199e5ccbb7b,Not Reported,TCGA-W3-AA1V-01Z-00-DX1,1,Primary Solid Tumor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9817ec15-605a-40db-b848-2199e5ccbb7b,Dead,3.506849,63.873973,1994.0,stage ii,"Skin, NOS",Lymph nodes of axilla or arm,Lymph,no,"Malignant melanoma, NOS",3.506849,c3eb5853-b803-4b10-81a0-585f761a3e18,584e9078-ff46-4f7e-92a4-055836498fda,1.665154e+08,9817ec15-605a-40db-b848-2199e5ccbb7b,TOP,TCGA-W3-AA1V-06B-01-TS1,6,Metastatic,0.0,100.0,0.0,2.0,0.0,0.0,0.0,80.0,NaN
2,c59a5615-2dd1-4902-a62b-0e312eab5484,Alive,NaN,75.523288,2005.0,stage iii,"Skin, NOS","Lymph node, NOS",Lymph,no,"Malignant melanoma, NOS",7.978082,057033de-f7d3-489c-aeb5-c2b71a730d84,d950dd14-9025-440b-9106-5970de364a40,9.343524e+08,c59a5615-2dd1-4902-a62b-0e312eab5484,Not Reported,TCGA-D3-A1QB-06Z-00-DX1,6,Metastatic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.978082
3,c59a5615-2dd1-4902-a62b-0e312eab5484,Alive,NaN,75.523288,2005.0,stage iii,"Skin, NOS","Lymph node, NOS",Lymph,no,"Malignant melanoma, NOS",7.978082,4e893fc1-69ff-4c5a-a9b4-8f9219613db0,79536980-88a8-4df9-aa4b-285f6bdaed22,4.829648e+08,c59a5615-2dd1-4902-a62b-0e312eab5484,TOP,TCGA-D3-A1QB-06A-01-TSA,6,Metastatic,14.0,85.0,0.0,3.0,1.0,0.0,1.0,85.0,7.978082
4,57882f61-4a32-49f1-afd4-61c70b184d8b,Alive,NaN,55.572603,2005.0,stage iii,"Skin, NOS","Lymph node, NOS",Lymph,no,"Malignant melanoma, NOS",4.430137,d7b82c68-db47-441b-88dc-b7fcd0c24d02,c7b0b6dc-e550-4ddc-b4c1-661eebb9fa07,3.576689e+08,57882f61-4a32-49f1-afd4-61c70b184d8b,TOP,TCGA-FS-A1ZT-06A-01-TSA,6,Metastatic,7.0,90.0,0.0,0.0,3.0,0.0,0.0,85.0,4.430137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,b1739cb5-09bc-414a-8f05-c66d2cb9868d,Dead,29.780822,51.624658,1978.0,i/ii nos,"Skin, NOS","Skin, NOS",Skin,no,"Malignant melanoma, NOS",29.780822,4d0a3fdc-815e-4a73-8b64-ffbdffa8a3df,77b41cfe-c186-47a8-ac8f-2e5a20ec17ba,1.461685e+09,b1739cb5-09bc-414a-8f05-c66d2cb9868d,Not Reported,TCGA-FS-A1ZC-06Z-00-DX2,6,Metastatic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
804,b1739cb5-09bc-414a-8f05-c66d2cb9868d,Dead,29.780822,51.624658,1978.0,i/ii nos,"Skin, NOS","Skin, NOS",Skin,no,"Malignant melanoma, NOS",29.780822,8cf99dab-74e9-4498-b92c-119ec19b7c44,2e2ddc8e-f6f4-48f4-9549-7b4198307860,2.340166e+08,b1739cb5-09bc-414a-8f05-c66d2cb9868d,TOP,TCGA-FS-A1ZC-06A-01-TSA,6,Metastatic,10.0,85.0,0.0,3.0,5.0,0.0,1.0,90.0,NaN
805,c787c4da-c564-44f1-89eb-dd9da107acb1,Alive,NaN,69.964384,2012.0,stage iib,"Skin, NOS",Skin of trunk,Skin,no,"Malignant melanoma, NOS",0.221918,78c7eafb-caeb-4e39-bfd6-772e093bd9f3,d8eb66a8-1cb9-451c-b3a5-8c037a1240fd,6.468050e+08,c787c4da-c564-44f1-89eb-dd9da107acb1,Not Reported,TCGA-EB-A44O-01Z-00-DX1,1,Primary Solid Tumor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.221918
806,c787c4da-c564-44f1-89eb-dd9da107acb1,Alive,NaN,69.964384,2012.0,stage iib,"Skin, NOS",Skin of trunk,Skin,no,"Malignant melanoma, NOS",0.221918,e274f684-1562-48d6-86a5-e650046d9a22,9ed87f0b-a49b-4141-9e90-1f9fe82c906c,4.945901e+08,c787c4da-c564-44f1-89eb-dd9da107acb1,Not Reported,TCGA-EB-A44O-06Z-00-DX1,6,Metasta

In [7]:
#Get Lymph nodes 
#Split into training and validation 
#Split each training and validation into further sub-labels: <3 years and >3 years 

In [8]:
df_lymph_nodes = df[df.biopsy_tissue_type == "Lymph"]

In [9]:
train_lymph_nodes, validation_lymph_nodes = np.split(df_lymph_nodes.sample(frac=1, random_state=42), 
                       [int(.8*len(df_lymph_nodes))])

In [10]:
train_lymph_nodes_early = train_lymph_nodes[train_lymph_nodes.years_survived <= 3]
train_lymph_nodes_late =  train_lymph_nodes[train_lymph_nodes.years_survived > 3]
validation_lymph_nodes_early = validation_lymph_nodes[validation_lymph_nodes.years_survived <= 3]
validation_lymph_nodes_late = validation_lymph_nodes[validation_lymph_nodes.years_survived > 3]

print(train_lymph_nodes_early.shape[0])
print(train_lymph_nodes_late.shape[0])
print(validation_lymph_nodes_early.shape[0])
print(validation_lymph_nodes_late.shape[0])
print(train_lymph_nodes.shape[0])
print(validation_lymph_nodes.shape[0])

120
192
28
51
312
79


In [11]:
lymph_training_early = train_lymph_nodes_early["slides__file_id"].to_list()
lymph_training_late = train_lymph_nodes_late["slides__file_id"].to_list()

lymph_validation_early = validation_lymph_nodes_early["slides__file_id"].to_list()
lymph_validation_late = validation_lymph_nodes_late["slides__file_id"].to_list()

print(lymph_training_early)
print(lymph_training_late)
print(lymph_validation_early)
print(lymph_validation_late)

['2536bc10-9fce-4ca7-9745-f15c96bce10d', '2774b738-3f6c-420a-aa1e-f7fcf527097a', '3c5314b4-47d0-4cac-92de-55a27322afae', '7f6f5106-ae56-482d-9dc8-0fe1f3162f9e', '4e717887-8630-4c0e-83e7-5de145146faa', '38b991fa-f216-4868-b4f1-b6183263c6c8', '0cf1123e-3359-4889-8cca-6b97afef0834', '9a93d3c3-0552-4f93-aace-fd6c0393272c', '550eeb48-be7f-4c70-9793-d26f2b100a76', '9bba7325-e476-4119-beb8-51259b4cec92', '6f523ee3-e71c-4c7e-a9de-47f8ab119ebe', 'a31090b1-b5a6-447d-ab01-a3c73de29a92', 'da168ddd-57e5-4139-9271-2b22068ea278', 'a5e701a7-5d36-4474-9c7e-e86741190da5', 'fde27372-8dc2-42ae-8bcf-5250b0664cac', '1a500d64-ee11-4967-88cb-19cf5efaa7c5', 'dc960c3c-f493-4254-8447-a5cfff674aaa', '008e1882-c9cc-4ad3-9e31-42f8790719ca', 'de6e04de-3aa3-46ab-9964-ccbc4b8b131a', 'f917c05d-353c-4b7f-9661-4933d14bd385', '41033e82-c2d4-4178-a0f0-54352a2d723e', '6a1cf713-8486-4dbd-8264-419306f41fad', '084d3e41-2125-4c2c-90f0-62e9722b01ec', '3344753b-2d0d-4708-a119-cebacc7af80d', '1c76d23e-938f-4112-8832-b9899c41c792',

In [12]:
#Get Skin 
#Split into training and validation 
#Split each training and validation into further sub-labels: <5 years and >5 years 

In [13]:
df_skin = df[df.biopsy_tissue_type == "Skin"]

In [14]:
train_skin, validation_skin = np.split(df_skin.sample(frac=1, random_state=42), 
                       [int(.8*len(df_skin))])

In [15]:
#Label 1: early, Label 2: late
train_skin_early = train_skin[train_skin.years_survived <= 3]
train_skin_late =  train_skin[train_skin.years_survived > 3]
validation_skin_early = validation_skin[validation_skin.years_survived <= 3]
validation_skin_late = validation_skin[validation_skin.years_survived > 3]

print(train_skin_early.shape[0])
print(train_skin_late.shape[0])
print(validation_skin_early.shape[0])
print(validation_skin_late.shape[0])
print(train_skin.shape[0])
print(validation_skin.shape[0])

124
47
34
9
171
43


In [16]:
skin_training_early = train_skin_early["slides__file_id"].to_list()
skin_training_late = train_skin_late["slides__file_id"].to_list()

skin_validation_early = validation_skin_early["slides__file_id"].to_list()
skin_validation_late = validation_skin_late["slides__file_id"].to_list()

print(skin_training_early)
print(skin_training_late)
print(skin_validation_early)
print(skin_validation_late)

['86789f1c-763c-4eec-94ba-32ea46661770', 'ed1af3d0-a609-4d86-ad80-5e15e6d48683', 'de69c852-f624-4367-a9ff-13a336228c8c', '88186c36-6c50-4a42-8dbd-cf76e3fc0ba6', '862dc232-cc6f-40a4-9da5-a8fe5666975e', '8218f730-2e47-47ca-983d-3c62a335b1a0', 'b79e09de-9300-422b-a62c-1cacc53bd6b9', 'ee40290e-5caf-4861-a005-650b6fe8501b', 'a1784a18-75eb-4dbe-b750-e945e2266264', 'eb1cce0b-cd30-4ffb-8296-3e418da95d56', 'a178524a-c343-483a-aa23-a56eaf8c4efd', 'c1c7228b-6661-4158-a078-24ca1f8d1ea7', '97fcb3a8-e213-4833-8da2-6a9030c475b5', '58ac66fb-80e9-4edf-8346-cff2e4d7cf80', '336d00ab-536e-4cb9-93b4-bd97076e1879', 'd8eb66a8-1cb9-451c-b3a5-8c037a1240fd', '72887cb9-afb7-4061-9840-01d68d82eacd', 'ea8d2a12-0a8b-4f30-b8a5-84f03b14dc4c', '81c239b5-7c34-4f44-ac17-2482220d767c', 'c177f308-b275-4550-bd9b-cda4a46e0c8c', '8a999396-ea3d-4203-84cb-de13f2907879', 'a444b1f6-aa65-4c3b-b445-185a2954db8e', '37e7e0bc-b935-4627-b279-6321f3737db9', '264f9d83-2973-494f-84c4-9b6acf06ca0c', '53fc4724-9a1e-44a4-a7bf-0fca600b934b',

In [17]:
base_dir = "/content/drive/MyDrive/Data/"

In [20]:
'''Alternative method using OpenSlide Deepzoom'''

#Calculate percentage of tissue in image 
def calculate_tissue_percentage(img, threshold):
    img = img.copy().convert("L")   #convert to grayscale
    tissue_pixels = 0
    width, height = img.width, img.height
    total_pixels = width*height
    for x in range(width):
        for y in range(height):
            if (img.getpixel((x,y)) < threshold):     #threshold value to reject white space
                tissue_pixels += 1
    return (tissue_pixels / total_pixels) * 100

#Information on images
def slide_info_dzi(dimension, image):
  slide = openslide.OpenSlide(os.path.join(base_dir, str(image) + ".svs")) #create svs image
  tiles = openslide.deepzoom.DeepZoomGenerator(slide, dimension, 0, False) #create deepzoom object 

  level_count = tiles.level_count - 1  #Number of levels
  tile_count = tiles.tile_count        #Number of tiles
  level_tiles = tiles.level_tiles      #Number of tiles per level
  
  return level_count, level_tiles

#Crop function to create tiles and save to directory
def crop_dzi(dimension, image, level, directory): 
  slide = openslide.OpenSlide(os.path.join(base_dir, str(image) + ".svs")) #create svs image
  tiles = openslide.deepzoom.DeepZoomGenerator(slide, dimension, 0, False) #create deepzoom object 

  #create RGB tiles and filter using threshold value
  tiles_rgb = []
  loop = 0
  counter = 0

  level_count, level_tiles = slide_info_dzi(dimension, image)  
  for x in range(level_tiles[level_count - level][0]):
    for y in range(level_tiles[level_count - level][1]):
      tile_rgb = tiles.get_tile((level_count - level), (x, y))    #get tile at each column/row
      #print(calculate_tissue_percentage(tile_rgb, 200))
      if(calculate_tissue_percentage(tile_rgb, 200) >= 35):       #calculate tissue percentage
        tiles_rgb.append(tile_rgb)                                #list to track number of tiles kept
        tile_rgb.save(directory + str(image) + "_" + str(counter) + ".png")     #png image saved to directory
        counter += 1
      #print("Loop" + str(loop))
      loop += 1
  return tiles_rgb


'''image_list = ['8cfab43c-09fa-408e-84eb-f61ba2841ebc']
image = '8cfab43c-09fa-408e-84eb-f61ba2841ebc'
slide_info_dzi(512, image)'''

"image_list = ['8cfab43c-09fa-408e-84eb-f61ba2841ebc']\nimage = '8cfab43c-09fa-408e-84eb-f61ba2841ebc'\nslide_info_dzi(512, image)"

In [21]:
def generate_tiles(directory, image_list):
  dimension = 512
  level = 3
  for image in image_list:
    tiles_rgb = crop_dzi(dimension, image, level, directory)
    print("List length" + str(len(tiles_rgb)))
  
#generate_tiles("/content/drive/MyDrive/Directory/", image_list)

In [ ]:
#skin_training_early - skin_train_dir -> early
#skin_training_late - skin_train_dir -> late
#skin_validation_early - skin_validation_dir -> early
#skin_validation_late - skin_validation_dir -> late
#do same for lymph

#shutil.move("/content/drive/MyDrive/readable_data.csv", "/content/drive/MyDrive/Directory/readable_data.csv")

base_dir_image = "/content/drive/MyDrive/Data"

#directory with early training skin images 
#directory with early validation skin images
#directory with late training late images 
#directory with late validation skin images 
generate_tiles("/content/drive/MyDrive/Data/skin_train_dir/early/", skin_training_early)
generate_tiles("/content/drive/MyDrive/Data/skin_train_dir/late/", skin_training_late)
generate_tiles("/content/drive/MyDrive/Data/skin_validation_dir/early/", skin_validation_early)
generate_tiles("/content/drive/MyDrive/Data/skin_validation_dir/late/", skin_validation_late)

#directory with early training skin images 
#directory with early validation skin images
#directory with late training late images 
#directory with late validation skin images 
generate_tiles("/content/drive/MyDrive/Data/lymph_train_dir/early/", lymph_training_early)
generate_tiles("/content/drive/MyDrive/Data/lymph_train_dir/late/", lymph_training_late)
generate_tiles("/content/drive/MyDrive/Data/lymph_validation_dir/early/", lymph_validation_early)
generate_tiles("/content/drive/MyDrive/Data/lymph_validation_dir/late/", lymph_validation_late)